In [1]:
# Instalar numpy
!pip install numpy==1.23.5


In [2]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 5.6 MB/s eta 0:00:00


In [3]:
import h2o
from h2o.automl import H2OAutoML

# Iniciar el entorno de H2O
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.27" 2025-04-15; OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpxq21pn4x
  JVM stdout: /tmp/tmpxq21pn4x/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpxq21pn4x/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,2 months and 12 days
H2O_cluster_name:,H2O_from_python_unknownUser_5o9v5a
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

# Cargar dataset limpio
ruta = '/content/drive/MyDrive/Prediccion de Diabetes - Proyecto Analitica/proyecto-diabetes-analitica/Datos/dataset_limpio.csv'
df = pd.read_csv(ruta)

# Convertir a formato H2O
df_h2o = h2o.H2OFrame(df)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
# Definir Variable objetivo
y = 'diabetes'

# Variables predictoras (todas menos la de salida)
X = [col for col in df.columns if col != y]

# Asegurar que la variable objetivo sea categórica
df_h2o[y] = df_h2o[y].asfactor()


In [7]:
#División del dataset
train, test = df_h2o.split_frame(ratios=[0.8], seed=42)


In [8]:
# Entrenamiento de modelos con H2O AutoML
aml = H2OAutoML(max_runtime_secs=300, seed=42, balance_classes=True)
aml.train(x=X, y=y, training_frame=train)


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/8
# GBM base models (used / total),1/4
# XGBoost base models (used / total),2/2
# GLM base models (used / total),1/1
# DRF base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [9]:
# Evaluar el mejor modelo
# Mostrar leaderboard (mejores modelos)
lb = aml.leaderboard
lb.head()

# Evaluar en el test set
perf = aml.leader.model_performance(test_data=test)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.021738670398908488
RMSE: 0.1474403960890925
LogLoss: 0.07847161953243063
AUC: 0.9795174499293514
AUCPR: 0.8862611688321732
Gini: 0.9590348998587028
Null degrees of freedom: 20032
Residual degrees of freedom: 20027
Null deviance: 11378.253240142136
Residual deviance: 3144.043908186366
AIC: 3156.043908186366

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5832675010211007
       0      1     Error    Rate
-----  -----  ----  -------  ---------------
0      18362  26    0.0014   (26.0/18388.0)
1      484    1161  0.2942   (484.0/1645.0)
Total  18846  1187  0.0255   (510.0/20033.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.583268     0.819915  88
max f2                       0.167654     0.789857  228
max f0point5                 0.778926     0.91668   65
max accuracy                 0.619339     0.974592  82
max precision                0.997689     1         0
max recall                   0.00191314   1         389
max specificity              0.997689     1         0
max absolute_mcc             0.619339     0.819478  82
max min_per_class_accuracy   0.100279     0.914238  272
max mean_per_class_accuracy  0.0814896    0.914583  287
max tns                      0.997689     18388     0
max fns                      0.997689     1630      0
max fps                      0.000152534  18388     399
max tps                      0.00191314   1645      389
max tnr                      0.997689     1         0
max fnr                      0.997689     0.990881  0
max fpr                      0.000152534  1         399
max tpr                      0.00191314   1         389

Gains/Lift Table: Avg response rate:  8.21 %, avg score:  8.55 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100334                   0.992167           12.1781    12.1781            1                0.994787     1                           0.994787            0.122188        0.122188                   1117.81   1117.81            0.122188
2        0.020017                    0.983375           12.1781    12.1781            1                0.987929     1                           0.991367            0.121581        0.243769                   1117.81   1117.81            0.243769
3        0.0300005                   0.974887           12.1781    12.1781            1                0.979144     1                           0.987299            0.121581        0.36535                    1117.81   1117.81            0.36535
4        0.0400339                   0.96387            12.1781    12.1781            1                0.969673     1                           0.982882            0.122188        0.487538                   1117.81   1117.81            0.487538
5        0.0500175                   0.946258           12.1781    12.1781            1                0.955835     1                           0.977483            0.121581        0.609119                   1117.81   1117.81            0.609119
6        0.100035                    0.216091           4.25383    8.21597            0.349301         0.423387     0.674651                    0.700435            0.212766        0.821884                   325.383   721.597            0.786427
7        0.150002                    0.104738           1.70323    6.04651            0.13986          0.152629

In [10]:
# guardar el modelo
modelo_guardado = h2o.save_model(model=aml.leader, path="/content/drive/MyDrive/Prediccion de Diabetes - Proyecto Analitica/proyecto-diabetes-analitica/Modelos", force=True)
print("Modelo guardado en:", modelo_guardado)



Modelo guardado en: /content/drive/MyDrive/Prediccion de Diabetes - Proyecto Analitica/proyecto-diabetes-analitica/Modelos/StackedEnsemble_AllModels_1_AutoML_1_20250608_225845
